In [1]:
import pandas as pd
import psycopg2
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus

In [2]:
tf_user = os.environ.get("TRIALFIRE_USER")
tf_pass = os.environ.get("TRIALFIRE_PASS")
tf_db = os.environ.get("TRIALFIRE_DB")
tf_host = os.environ.get("TRIALFIRE_HOST")
tf_api = os.environ.get("TRIALFIRE_API")
tf_port = 5432

In [3]:
uri = f"postgresql+psycopg2://{quote_plus(tf_user)}:{quote_plus(tf_pass)}@{tf_host}:{tf_port}/{tf_db}"
alchemyEngine = create_engine(uri)

In [4]:
dbConnection = alchemyEngine.connect();

## Calculate and Set Class Balances Based on 10% of Data

In [5]:
# Us this to get the balance ratios to limit the number of rows
true_counts = """
SELECT COUNT(*)
FROM person_""" + tf_api + """ P
WHERE (P.has_purchased is True) 
AND (P.first_seen < CURRENT_DATE - INTERVAL '7 days')
AND (P.first_seen > CURRENT_DATE - INTERVAL '6 months');
"""

In [6]:
# Us this to get the balance ratios to limit the number of rows
false_counts = """
SELECT COUNT(*)
FROM person_""" + tf_api + """ P
WHERE (P.has_purchased is False) 
AND (P.first_seen < CURRENT_DATE - INTERVAL '7 days') 
AND (P.first_seen > CURRENT_DATE - INTERVAL '6 months');
"""

In [7]:
df_true = pd.read_sql(true_counts, dbConnection);
df_true.iloc[0,0]

76953

In [8]:
df_false = pd.read_sql(false_counts, dbConnection);
df_false.iloc[0,0]

2600466

In [9]:
lim_true = int(df_true.iloc[0,0] * .1)
lim_true

7695

In [10]:
lim_false = int(df_false.iloc[0,0] * .1)
lim_false

260046

## Get the Data

In [11]:
person_purchased = """
SELECT
P.person_id, 
P.first_utm_medium,
P.first_utm_source,
P.multi_device::int, 
P.session_count,
P.last_utm_medium,
P.last_utm_source,
P.source_category,
P.source_category_2,
P.source_category_3, 
EXTRACT(EPOCH FROM (P.last_seen - P.first_seen)) as seconds_since_first_vist,
count(distinct S.first_utm_content) as first_utm_content_distinct,	
count(distinct S.first_utm_medium) as first_utm_medium_distinct,	
count(distinct S.first_utm_source) as first_utm_source_distinct,	
count(distinct S.first_utm_term) as first_utm_term_distinct,
sum(S.click_count) as click_count_sum,
sum(S.input_count) as input_count_sum,
sum(S.identify_count) as identify_count_sum,
sum(S.view_count) as view_count_sum,
sum(S.page_count) page_count_sum,
count(distinct S.source_category) as source_category_distinct,
avg(S.session_duration) as session_duration_avg,
P.has_purchased::int
FROM person_""" + tf_api + """ P
JOIN session_""" + tf_api + """ S 
ON S.person_id = P.person_id
WHERE (P.has_purchased is True) 
AND (P.first_seen < CURRENT_DATE - INTERVAL '7 days')
AND (P.first_seen > CURRENT_DATE - INTERVAL '6 months')
GROUP BY P.person_id
ORDER BY random()
LIMIT """ + str(lim_true) + ';'

In [12]:
person_not = """
SELECT
P.person_id, 
P.first_utm_medium,
P.first_utm_source,
P.multi_device::int, 
P.session_count,
P.last_utm_medium,
P.last_utm_source,
P.source_category,
P.source_category_2,
P.source_category_3, 
EXTRACT(EPOCH FROM (P.last_seen - P.first_seen)) as seconds_since_first_vist,
count(distinct S.first_utm_content) as first_utm_content_distinct,	
count(distinct S.first_utm_medium) as first_utm_medium_distinct,	
count(distinct S.first_utm_source) as first_utm_source_distinct,	
count(distinct S.first_utm_term) as first_utm_term_distinct,
sum(S.click_count) as click_count_sum,
sum(S.input_count) as input_count_sum,
sum(S.identify_count) as identify_count_sum,
sum(S.view_count) as view_count_sum,
sum(S.page_count) page_count_sum,
count(distinct S.source_category) as source_category_distinct,
avg(S.session_duration) as session_duration_avg,
P.has_purchased::int
FROM person_""" + tf_api + """ P
JOIN session_""" + tf_api + """ S 
ON S.person_id = P.person_id
WHERE (P.has_purchased is False) 
AND (P.first_seen < CURRENT_DATE - INTERVAL '7 days')
AND (P.first_seen > CURRENT_DATE - INTERVAL '6 months')
GROUP BY P.person_id
ORDER BY random()
LIMIT  """ + str(lim_false) + ';'

In [13]:
person_test = """
SELECT
P.person_id, 
P.first_utm_medium,
P.first_utm_source,
P.multi_device::int, 
P.session_count,
P.last_utm_medium,
P.last_utm_source,
P.source_category,
P.source_category_2,
P.source_category_3, 
EXTRACT(EPOCH FROM (P.last_seen - P.first_seen)) as seconds_since_first_vist,
count(distinct S.first_utm_content) as first_utm_content_distinct,	
count(distinct S.first_utm_medium) as first_utm_medium_distinct,	
count(distinct S.first_utm_source) as first_utm_source_distinct,	
count(distinct S.first_utm_term) as first_utm_term_distinct,
sum(S.click_count) as click_count_sum,
sum(S.input_count) as input_count_sum,
sum(S.identify_count) as identify_count_sum,
sum(S.view_count) as view_count_sum,
sum(S.page_count) page_count_sum,
count(distinct S.source_category) as source_category_distinct,
avg(S.session_duration) as session_duration_avg,
P.has_purchased::int
FROM person_""" + tf_api + """ P
JOIN session_""" + tf_api + """ S 
ON S.person_id = P.person_id
WHERE (P.has_purchased is False) 
AND (P.last_seen > CURRENT_DATE - INTERVAL '7 days')
GROUP BY P.person_id;
"""

In [14]:
df_purchased = pd.read_sql(person_purchased, dbConnection);

In [15]:
df_not = pd.read_sql(person_not, dbConnection);

In [16]:
df_test = pd.read_sql(person_test, dbConnection);

In [17]:
print(df_purchased.shape)
print(df_not.shape)
print(df_test.shape)

(7695, 23)
(260046, 23)
(251630, 23)


In [18]:
df_purchased.to_pickle("person_purchased.pkl")
df_not.to_pickle("person_not.pkl")
df_test.to_pickle("person_test.pkl")


In [ ]:
dbConnection.close()